### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 1

today = date.today()
today

datetime.date(2022, 5, 11)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022 AND quarter <= 1) 
OR (year = 2022-1 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(106, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(229, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
12,BCP,2022,Q1,9696604,-22799,9719403,"42,630.83"
47,KSL,2022,Q1,639300,13859,625441,"4,512.89"
22,CPNREIT,2022,Q1,869788,-34503,904291,"2,620.91"
9,BANPU,2022,Q1,18580896,-1963881,20544777,"1,046.13"
88,THG,2022,Q1,2079171,-235059,2314230,984.53


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

80

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

103

In [8]:
for rcd in rcds:
    print(rcd)

['ACE', 2022, 'Q1', 1468062, 1274421, 193641, 15.19, 698]
['ADVANC', 2022, 'Q1', 26589096, 27322055, -732959, -2.68, 6]
['AJ', 2022, 'Q1', 467710, 692383, -224673, -32.45, 12]
['AOT', 2022, 'Q1', -17151697, -6456020, -10695677, -165.67, 24]
['AP', 2022, 'Q1', 4870212, 5010831, -140619, -2.81, 25]
['ASIAN', 2022, 'Q1', 1077825, 928093, 149732, 16.13, 36]
['ASK', 2022, 'Q1', 1277644, 958779, 318865, 33.26, 38]
['ASP', 2022, 'Q1', 775000, 741274, 33726, 4.55, 40]
['AWC', 2022, 'Q1', 2100590, -2583450, 4684040, 181.31, 699]
['BANPU', 2022, 'Q1', 18580896, -1963881, 20544777, 1046.13, 47]
['BAY', 2022, 'Q1', 34707618, 22512053, 12195565, 54.17, 49]
['BBL', 2022, 'Q1', 26702007, 16433170, 10268837, 62.49, 50]
['BCP', 2022, 'Q1', 9696604, -22799, 9719403, 42630.83, 52]
['BCPG', 2022, 'Q1', 2850502, 1861919, 988583, 53.09, 53]
['BDMS', 2022, 'Q1', 10040447, 5984852, 4055595, 67.76, 54]
['BFIT', 2022, 'Q1', 696050, 1435683, -739633, -51.52, 58]
['BH', 2022, 'Q1', 1849587, 530070, 1319517, 248.9

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
12,BCP,2022,Q1,9696604,-22799,9719403,"42,630.83"
47,KSL,2022,Q1,639300,13859,625441,"4,512.89"
22,CPNREIT,2022,Q1,869788,-34503,904291,"2,620.91"
9,BANPU,2022,Q1,18580896,-1963881,20544777,"1,046.13"
88,THG,2022,Q1,2079171,-235059,2314230,984.53


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,BH,2022,Q1,1849587,530070,1319517,248.93
67,RJH,2022,Q1,1377891,436349,941542,215.78
83,SVI,2022,Q1,1521472,608053,913419,150.22
54,MAKRO,2022,Q1,14003291,6615551,7387740,111.67
64,PTTGC,2022,Q1,39499189,18678590,20820599,111.47


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
12,BCP,2022,Q1,9696604,-22799,9719403,"42,630.83"
47,KSL,2022,Q1,639300,13859,625441,"4,512.89"
22,CPNREIT,2022,Q1,869788,-34503,904291,"2,620.91"
9,BANPU,2022,Q1,18580896,-1963881,20544777,"1,046.13"
88,THG,2022,Q1,2079171,-235059,2314230,984.53


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,BH,2022,Q1,1849587,530070,1319517,248.93
67,RJH,2022,Q1,1377891,436349,941542,215.78
83,SVI,2022,Q1,1521472,608053,913419,150.22
54,MAKRO,2022,Q1,14003291,6615551,7387740,111.67
64,PTTGC,2022,Q1,39499189,18678590,20820599,111.47
44,KCE,2022,Q1,2512732,1205287,1307445,108.48
79,SPALI,2022,Q1,7507057,4242353,3264704,76.96
78,SNC,2022,Q1,765084,434356,330728,76.14
32,GGC,2022,Q1,722960,415739,307221,73.90
14,BDMS,2022,Q1,10040447,5984852,4055595,67.76


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2022,Q1,1468062,1274421,193641,15.19
5,ASIAN,2022,Q1,1077825,928093,149732,16.13
6,ASK,2022,Q1,1277644,958779,318865,33.26
10,BAY,2022,Q1,34707618,22512053,12195565,54.17
11,BBL,2022,Q1,26702007,16433170,10268837,62.49
13,BCPG,2022,Q1,2850502,1861919,988583,53.09
14,BDMS,2022,Q1,10040447,5984852,4055595,67.76
16,BH,2022,Q1,1849587,530070,1319517,248.93
18,BPP,2022,Q1,5011201,3761295,1249906,33.23
19,COTTO,2022,Q1,608126,482435,125691,26.05
